In [1]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle

# set verbose=0 the skip the charts and other information
# set verbose=1 to see chart and other information
verbose=0
    

In [2]:
model_dir = os.path.join(os.getcwd(), 'model')
data_dir = os.path.join(os.getcwd(), 'data')
model_filename = 'model.pkl'
scaler_filename = 'scaler.pkl'
testdata_file = 'datatest.txt'

# load model and scaler
with open(os.path.join(model_dir, model_filename), 'rb') as model_file:
    model = pickle.load(model_file)
    
with open(os.path.join(model_dir, scaler_filename), 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)

Using TensorFlow backend.












Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [3]:
test_df = pd.read_csv(os.path.join(data_dir, testdata_file))
test_df['date']= pd.to_datetime(test_df['date']) 
test_df.sort_values(by='date', inplace=True, ascending=True)
test_df.reset_index(drop=True)

if verbose==1:
    print('Training Data:')
    print('The time series starts from: ', test_df.date.min())
    print('The time series ends on: ', test_df.date.max())
    print()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,2015-02-02 14:19:00,23.700000,26.272000,585.200000,749.200000,0.004764,1
1,2015-02-02 14:19:59,23.718000,26.290000,578.400000,760.400000,0.004773,1
2,2015-02-02 14:21:00,23.730000,26.230000,572.666667,769.666667,0.004765,1
3,2015-02-02 14:22:00,23.722500,26.125000,493.750000,774.750000,0.004744,1
4,2015-02-02 14:23:00,23.754000,26.200000,488.600000,779.000000,0.004767,1
5,2015-02-02 14:23:59,23.760000,26.260000,568.666667,790.000000,0.004779,1
6,2015-02-02 14:25:00,23.730000,26.290000,536.333333,798.000000,0.004776,1
7,2015-02-02 14:25:59,23.754000,26.290000,509.000000,797.000000,0.004783,1
8,2015-02-02 14:26:59,23.754000,26.350000,476.000000,803.200000,0.004794,1
9,2015-02-02 14:28:00,23.736000,26.390000,510.000000,809.000000,0.004796,1


In [4]:
import mycharts

if verbose==1:
    mycharts.chart01(test_df.columns[1:], test_df, 'blue')

In [5]:
X = test_df.loc[:, 'Temperature' : 'HumidityRatio'].copy().to_numpy()
y = test_df.loc[:, 'Occupancy'].copy().to_numpy()

In [6]:
X = scaler.transform(X)

#reshape for LSTM samples, 1 timestep and 5 features
X = X.reshape(-1, 1, 5)


In [7]:
pred = model.predict(X)
pred_classes = model.predict_classes(X)

In [8]:
pred_classes_squeezed = np.squeeze(pred_classes)


In [9]:
from sklearn.metrics import classification_report, confusion_matrix

print(pd.crosstab(y, pred_classes_squeezed, rownames=['True'], colnames=['Predicted'], margins=True))
print()
print(classification_report(y, pred_classes_squeezed))

Predicted     0     1   All
True                       
0          1638    55  1693
1             3   969   972
All        1641  1024  2665

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      1693
           1       0.95      1.00      0.97       972

    accuracy                           0.98      2665
   macro avg       0.97      0.98      0.98      2665
weighted avg       0.98      0.98      0.98      2665

